In [1]:
# library needed
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import string
import re

In [2]:
dat = pd.read_csv("./job_training_data.csv")

In [3]:
# main parsing function
# dat: the raw pandas dataframe
# stop_words: a set of words to omitt when parsing
def parsing(dat, stop_words):
    
    # let character rows be merged into one single text (only for columns of type 'object')
    data = dat.fillna('NaN')
    data_object_colnames = data.columns[data.dtypes == 'object']
    text = data[data_object_colnames].agg(' '.join, axis=1)
    
    # results for parsing
    vectorizer = CountVectorizer(stop_words=stop_words)
    result = pd.DataFrame.sparse.from_spmatrix(vectorizer.fit_transform(text))
    
    # merge parsed results, number of NA for each field, numeric columns (less fraud and job id)
    cleaned_data = pd.concat([result,
                              dat.isna().astype('int'),
                              dat.loc[:,dat.dtypes == 'int64'].iloc[:,1:4]], axis = 1, ignore_index=True)
    return cleaned_data
    

In [ ]:
stop_words = {"a", "the"}
result = parsing(dat, sto)

# Below are code for thought process (can ignore)

In [ ]:
# takes in a text
# gets rid of punctuation
def ridpunctuation(text):
    returnre.sub(r'[^\w\s]', '', text)

In [ ]:
# Method 1
# let character rows be merged into one single text
data = dat.fillna(' NaN ')
data_object_colnames = data.columns[data.dtypes == 'object']
text = data[data_object_colnames[0]]
for col in range(1,len(data_object_colnames)):
    text = text + data[data_object_colnames[col]]

In [ ]:
# Method 2
data = dat.fillna('NaN')
data_object_colnames = data.columns[data.dtypes == 'object']
text = data[data_object_colnames].agg(' '.join, axis=1)

In [ ]:
# stop_words
stop_words = {'a', 'the'}

In [ ]:
# results for parsing
vectorizer = CountVectorizer(stop_words=stop_words)
result = pd.DataFrame.sparse.from_spmatrix(vectorizer.fit_transform(text))

In [ ]:
# NA info 
cleaned_data = pd.concat([result ,dat.isna().astype('int'),dat.loc[:,dat.dtypes == 'int64'].iloc[:,1:4]],axis = 1,ignore_index=True)

# Below are scratch code (can ignore)

In [ ]:
test_series = pd.DataFrame({"a":['hello"s , hello hello,hi you are',"hello hello today's\xa0Document Hello"]})

In [ ]:
# save results of parsing
vectorizer = CountVectorizer()
result = vectorizer.fit_transform(test_series['a'])

In [ ]:
pd.DataFrame.sparse.from_spmatrix(result)

In [ ]:
vectorizer.vocabulary_

In [ ]:
test = CountVectorizer().fit_transform(data.iloc[:,1])
test_names
pd.DataFrame.sparse.from_spmatrix(test)

In [ ]:
def parsing_series(series):
    sparse_parse = CountVectorizer(stop_words=stop_words).fit_transform(series)
    pandas.DataFrame.sparse.from_spmatrix


In [ ]:
data

In [ ]:
def parsing(col):
    

In [ ]:
# concat na to 
pd.concat([data ,data.isna().astype('int')],axis = 1,ignore_index=True)

In [ ]:
pd.concat()